In [239]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from eda_py import EDA
from eALS_adaptor import eALSAdaptor
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from cv_py import CrossValidation
from visualize_py import Visualize

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [240]:
dl = DataLoader()

In [241]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [242]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [243]:
# initial split
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [244]:
cv = CrossValidation(train, 5)
cv_t = CrossValidation(train_t, 5)
validate, calibrate = cv.split_k_fold(22)
validate_t, calibrate_t = cv_t.split_k_fold(22)

In [245]:
factors = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
iterations = [22, 24, 26, 28, 30]

In [246]:
space_iALS = {'factors' : factors, 'regularization' : [90], 'alpha' : [0.3], 'iterations' : iterations}
hyper_ials_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')
hyper_ials_add.to_csv('curves_ials_add.csv')

In [247]:
space_iALS_t = {'factors' : factors, 'regularization' : [90], 'alpha' : [1.3], 'iterations' : iterations}
hyper_ials_t_add = cv_t.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')
hyper_ials_t_add.to_csv('curves_ials_t_add.csv')

In [248]:
space_eALS_t = {'factors' : factors, 'regularization' : [50], 'alpha' : [0.1], 'w0' : [5000], 'iterations' : iterations}
hyper_eals_t_add = cv_t.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_eALS_t, model_class='eALS')
hyper_eals_t_add.to_csv('curves_eals_add.csv')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [249]:
space_BPR = {'factors' : factors, 'regularization' : [0.03], 'learning_rate' : [0.005], 'iterations' : iterations}
hyper_bpr_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_BPR, model_class='BPR')
hyper_bpr_add.to_csv('curves_bpr_add.csv')

In [250]:
space_LMF = {'factors' : factors, 'regularization' : [40], 'learning_rate' : [1.2], 'iterations' : iterations, 'neg_prop': [50]}
hyper_lmf_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_LMF, model_class='LMF')
hyper_lmf_add.to_csv('curves_lmf_add.csv')

In [238]:
# saving locally
hyper_ials.to_csv('curves_ials.csv')
hyper_ials_t.to_csv('curves_ials_t.csv')
hyper_eals_t.to_csv('curves_eals.csv')
hyper_bpr.to_csv('curves_bpr.csv')
hyper_lmf.to_csv('curves_lmf.csv')